In [1]:
!pip install -q torch torchvision transformers datasets peft accelerate bitsandbytes
!pip install -q evaluate rouge-score sacrebleu matplotlib seaborn plotly
!pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 20.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 9.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 9.4 MB/s eta 0:00:00


In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

In [2]:
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,
    TrainingArguments, Trainer, DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, TaskType, PeftModel
from datasets import load_dataset, Dataset, concatenate_datasets
import json
import os
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

In [3]:
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
print(f"CUDA Available: {torch.cuda.is_available()}")

GPU: Tesla T4
CUDA Available: True


In [4]:
model_name = "microsoft/phi-2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [7]:
print(f"Parameters: {base_model.num_parameters():,}")

Parameters: 2,779,683,840


In [8]:
science_data = load_dataset("sciq", split="train[:300]")

In [9]:
science_data

Dataset({
    features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
    num_rows: 300
})

In [12]:
creative_data = load_dataset("roneneldan/TinyStories", split="train[:300]")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00004-2d5a1467fff108(…):   0%|          | 0.00/249M [00:00<?, ?B/s]

data/train-00001-of-00004-5852b56a2bd28f(…):   0%|          | 0.00/248M [00:00<?, ?B/s]

data/train-00002-of-00004-a26307300439e9(…):   0%|          | 0.00/246M [00:00<?, ?B/s]

data/train-00003-of-00004-d243063613e5a0(…):   0%|          | 0.00/248M [00:00<?, ?B/s]

data/validation-00000-of-00001-869c898b5(…):   0%|          | 0.00/9.99M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2119719 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/21990 [00:00<?, ? examples/s]

In [13]:
creative_data

Dataset({
    features: ['text'],
    num_rows: 300
})

In [14]:
finance_data = load_dataset("gbharti/finance-alpaca", split="train[:300]")

In [15]:
finance_data

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 300
})

In [16]:
def format_science_data(examples):
    """Format SciQ dataset: question + support + correct_answer"""
    texts = []
    for i in range(len(examples["question"])):
        question = examples["question"][i]
        support = examples["support"][i] if examples["support"][i] else "No additional context provided."
        correct_answer = examples["correct_answer"][i]

        # Create scientific Q&A format
        text = f"Scientific Question: {question}\n\nContext: {support}\n\nAnswer: {correct_answer}"
        texts.append(text)
    return {"text": texts}

In [17]:
def format_creative_data(examples):
    """Format TinyStories dataset: use full stories"""
    texts = []
    for i in range(len(examples["text"])):
        story = examples["text"][i]

        # Clean and format the story
        if len(story) > 50:
            # Split story into prompt and continuation for training
            sentences = story.split('. ')
            if len(sentences) > 2:
                prompt = sentences[0] + '.'
                continuation = '. '.join(sentences[1:])
                text = f"Creative Prompt: Continue this story: {prompt}\n\nStory: {continuation}"
            else:
                text = f"Creative Prompt: Write a creative story.\n\nStory: {story}"
            texts.append(text)
    return {"text": texts}

In [18]:
def format_finance_data(examples):
    """Format Finance-Alpaca dataset: instruction + input + output"""
    texts = []
    for i in range(len(examples["instruction"])):
        instruction = examples["instruction"][i]
        input_text = examples["input"][i] if examples["input"][i] else ""
        output = examples["output"][i]

        # Create financial instruction format
        if input_text:
            text = f"Financial Query: {instruction}\n\nContext: {input_text}\n\nFinancial Analysis: {output}"
        else:
            text = f"Financial Query: {instruction}\n\nFinancial Analysis: {output}"
        texts.append(text)
    return {"text": texts}

In [19]:
science_formatted = science_data.map(
    format_science_data,
    batched=True,
    remove_columns=science_data.column_names,
    desc="Formatting SciQ dataset"
)

creative_formatted = creative_data.map(
    format_creative_data,
    batched=True,
    remove_columns=creative_data.column_names,
    desc="Formatting TinyStories dataset"
)

finance_formatted = finance_data.map(
    format_finance_data,
    batched=True,
    remove_columns=finance_data.column_names,
    desc="Formatting Finance-Alpaca dataset"
)

Formatting SciQ dataset:   0%|          | 0/300 [00:00<?, ? examples/s]

Formatting TinyStories dataset:   0%|          | 0/300 [00:00<?, ? examples/s]

Formatting Finance-Alpaca dataset:   0%|          | 0/300 [00:00<?, ? examples/s]

In [20]:
science_formatted = science_formatted.filter(lambda x: len(x["text"]) > 100)
creative_formatted = creative_formatted.filter(lambda x: len(x["text"]) > 100)
finance_formatted = finance_formatted.filter(lambda x: len(x["text"]) > 100)

Filter:   0%|          | 0/300 [00:00<?, ? examples/s]

Filter:   0%|          | 0/300 [00:00<?, ? examples/s]

Filter:   0%|          | 0/300 [00:00<?, ? examples/s]

In [23]:
science_lora_config = LoraConfig(
    r=16,  # Higher rank for complex scientific reasoning
    lora_alpha=32,
    target_modules=["Wqkv", "out_proj", "fc1", "fc2"],  # Phi-2 specific modules
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

creative_lora_config = LoraConfig(
    r=12,  # Medium rank for creative patterns
    lora_alpha=24,
    target_modules=["Wqkv", "out_proj", "fc1", "fc2"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

finance_lora_config = LoraConfig(
    r=8,   # Lower rank for structured financial data
    lora_alpha=16,
    target_modules=["Wqkv", "out_proj", "fc1", "fc2"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

In [24]:
def train_specialized_adapter(dataset, lora_config, adapter_name, output_dir):
    print(f"\n🔥 Training {adapter_name} adapter...")

    # Create model with LoRA
    model = get_peft_model(base_model, lora_config)
    model.config.use_cache = False

    # Tokenization function
    def tokenize_function(examples):
        return tokenizer(
            examples["text"],
            truncation=True,
            padding=False,
            max_length=512,
            return_tensors=None
        )

    # Tokenize dataset
    tokenized_dataset = dataset.map(tokenize_function, batched=True, desc=f"Tokenizing {adapter_name}")

    # Training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        num_train_epochs=1,
        max_steps=30,  # Quick training for demo
        learning_rate=2e-4,
        fp16=True,
        optim="paged_adamw_8bit",
        logging_steps=5,
        save_strategy="no",
        report_to="none",
        dataloader_num_workers=0
    )

    # Data collator
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False
    )

    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
        data_collator=data_collator,
    )

    # Train
    train_result = trainer.train()

    # Save adapter
    model.save_pretrained(output_dir)

    # Cleanup
    del model, trainer
    torch.cuda.empty_cache()

    print(f"✅ {adapter_name} adapter training completed!")
    return train_result

In [25]:
science_result = train_specialized_adapter(science_formatted, science_lora_config, "Science", "./science_lora")


🔥 Training Science adapter...


Tokenizing Science:   0%|          | 0/300 [00:00<?, ? examples/s]

Step,Training Loss
5,2.095000
10,1.876100
15,1.730500
20,1.736000
25,1.584800
30,1.634600


✅ Science adapter training completed!


In [26]:
creative_result = train_specialized_adapter(creative_formatted, creative_lora_config, "Creative", "./creative_lora")


🔥 Training Creative adapter...


Tokenizing Creative:   0%|          | 0/300 [00:00<?, ? examples/s]

Step,Training Loss
5,1.825100
10,1.579700
15,1.323200
20,1.448200
25,1.401900
30,1.385300


✅ Creative adapter training completed!


In [27]:
finance_result = train_specialized_adapter(finance_formatted, finance_lora_config, "Finance", "./finance_lora")


🔥 Training Finance adapter...


Tokenizing Finance:   0%|          | 0/300 [00:00<?, ? examples/s]

Step,Training Loss
5,2.796700
10,2.731400
15,2.799500
20,2.807500
25,2.659700
30,2.426700


✅ Finance adapter training completed!


In [39]:
# Advanced LoRA composition and merging utilities
class AdvancedLoRAComposer:
    def __init__(self, base_model, tokenizer):
        self.base_model = base_model
        self.tokenizer = tokenizer
        self.adapters = {}
        self.current_composition = None

    def load_adapter(self, adapter_path, adapter_name):
        """Load a LoRA adapter"""
        print(f"Loading {adapter_name} adapter from {adapter_path}")
        try:
            adapter_model = PeftModel.from_pretrained(self.base_model, adapter_path)
            self.adapters[adapter_name] = adapter_model
            print(f"✅ {adapter_name} adapter loaded successfully!")
        except Exception as e:
            print(f"❌ Error loading {adapter_name} adapter: {e}")


    def test_single_adapter(self, adapter_name, prompt, max_tokens=100):
        """Test a single adapter"""
        if adapter_name not in self.adapters:
            return f"Adapter {adapter_name} not loaded!"

        model = self.adapters[adapter_name]
        model.eval()

        inputs = self.tokenizer(prompt, return_tensors="pt").to("cuda")

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_tokens,
                temperature=0.7,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id
            )

        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response[len(prompt):].strip()


    def analyze_adapter_performance(self, test_prompts):
        """Analyze performance across different adapters"""
        results = {}

        for adapter_name in self.adapters.keys():
            results[adapter_name] = {}
            for prompt_type, prompt in test_prompts.items():
                response = self.test_single_adapter(adapter_name, prompt, max_tokens=80)
                results[adapter_name][prompt_type] = {
                    'prompt': prompt ,
                    'response': response,
                    'response_length': len(response)
                }

        return results

In [40]:
composer = AdvancedLoRAComposer(base_model, tokenizer)

In [41]:
composer.load_adapter("./science_lora", "Science")
composer.load_adapter("./creative_lora", "Creative")
composer.load_adapter("./finance_lora", "Finance")

Loading Science adapter from ./science_lora
✅ Science adapter loaded successfully!
Loading Creative adapter from ./creative_lora
✅ Creative adapter loaded successfully!
Loading Finance adapter from ./finance_lora
✅ Finance adapter loaded successfully!


In [42]:
test_prompts = {
    "science": "Scientific Claim: Increased CO2 levels affect plant growth rates.",
    "creative": "Creative Prompt: The old lighthouse keeper discovered a mysterious...",
    "finance": "Financial Query: What factors should I consider before investing in renewable energy stocks?",
    "mixed": "Analyze the economic impact of climate change on agricultural sectors."
}

performance_results = composer.analyze_adapter_performance(test_prompts)

for prompt_type, prompt in test_prompts.items():
    print(f"\n📋 Test Type: {prompt_type.upper()}")
    print(f"Prompt: {prompt}")
    print("-" * 50)

    for adapter_name in ["Science", "Creative", "Finance"]:
        result = performance_results[adapter_name][prompt_type]
        print(f"\n{adapter_name} Adapter Response:")
        print(f"Length: {result['response_length']} chars")
        print(f"Content: {result['response']}")

    print("\n" + "=" * 60)


📋 Test Type: SCIENCE
Prompt: Scientific Claim: Increased CO2 levels affect plant growth rates.
--------------------------------------------------

Science Adapter Response:
Length: 390 chars
Content: Scientific Support: Studies have shown that elevated CO2 levels can actually stimulate plant growth rates. This is because CO2 is a key ingredient in photosynthesis, the process by which plants make food. Additionally, higher CO2 levels can lead to larger and more efficient photosynthetic cells.

Counterargument: While it is true that increased CO2 levels can stimulate plant growth, this

Creative Adapter Response:
Length: 416 chars
Content: Evidence: Numerous scientific studies have shown that increasing levels of CO2 in the atmosphere have a positive effect on plant growth. Plants require carbon dioxide to carry out photosynthesis, and increased levels of CO2 can increase the rate of photosynthesis. However, other factors such as temperature, water availability, and soil quality also pl

In [33]:
def merge_lora_weights(adapters_info, merge_strategy="weighted"):
    """
    Merge multiple LoRA adapters with different strategies
    adapters_info: dict with adapter_name: weight
    """

    if merge_strategy == "weighted":
        print("Weighted merge based on specified weights")
        for adapter, weight in adapters_info.items():
            print(f"   {adapter}: {weight*100:.1f}%")

    elif merge_strategy == "equal":
        print("Equal weight merge - each adapter contributes equally")
        equal_weight = 1.0 / len(adapters_info)
        adapters_info = {adapter: equal_weight for adapter in adapters_info.keys()}

    elif merge_strategy == "performance_based":
        print("Performance-based merge - higher weight for better performers")
        # Simulate performance-based weighting
        performance_weights = {"Science": 0.4, "Creative": 0.35, "Finance": 0.25}
        adapters_info = performance_weights

    return adapters_info

In [34]:
merging_experiments = [
    {
        "name": "Science-Focused Merge",
        "weights": {"Science": 0.6, "Creative": 0.2, "Finance": 0.2},
        "strategy": "weighted"
    },
    {
        "name": "Creative-Focused Merge",
        "weights": {"Science": 0.2, "Creative": 0.6, "Finance": 0.2},
        "strategy": "weighted"
    },
    {
        "name": "Equal Merge",
        "weights": {"Science": 1.0, "Creative": 1.0, "Finance": 1.0},
        "strategy": "equal"
    },
    {
        "name": "Performance-Based Merge",
        "weights": {},
        "strategy": "performance_based"
    }
]

print("🔬 Testing Different Merging Strategies")
print("=" * 50)

merging_results = {}

for experiment in merging_experiments:
    print(f"\n🧪 Experiment: {experiment['name']}")
    merged_weights = merge_lora_weights(experiment['weights'], experiment['strategy'])
    merging_results[experiment['name']] = merged_weights
    print(f"Final weights: {merged_weights}")

🔬 Testing Different Merging Strategies

🧪 Experiment: Science-Focused Merge
Weighted merge based on specified weights
   Science: 60.0%
   Creative: 20.0%
   Finance: 20.0%
Final weights: {'Science': 0.6, 'Creative': 0.2, 'Finance': 0.2}

🧪 Experiment: Creative-Focused Merge
Weighted merge based on specified weights
   Science: 20.0%
   Creative: 60.0%
   Finance: 20.0%
Final weights: {'Science': 0.2, 'Creative': 0.6, 'Finance': 0.2}

🧪 Experiment: Equal Merge
Equal weight merge - each adapter contributes equally
Final weights: {'Science': 0.3333333333333333, 'Creative': 0.3333333333333333, 'Finance': 0.3333333333333333}

🧪 Experiment: Performance-Based Merge
Performance-based merge - higher weight for better performers
Final weights: {'Science': 0.4, 'Creative': 0.35, 'Finance': 0.25}


In [43]:
# Updated domain keywords based on your actual datasets
class SmartAdapterRouter:
    def __init__(self, composer):
        self.composer = composer
        # Updated keywords to match your dataset domains
        self.domain_keywords = {
            "Science": [
                # SciQ-specific terms
                "scientific", "research", "experiment", "hypothesis", "theory", "analysis",
                "data", "study", "evidence", "method", "observation", "conclusion",
                "physics", "chemistry", "biology", "science", "laboratory", "test"
            ],
            "Creative": [
                # TinyStories-specific terms
                "story", "creative", "character", "plot", "narrative", "tale", "fiction",
                "once", "there", "little", "big", "happy", "sad", "adventure", "friend",
                "magical", "wonderful", "beautiful", "scary", "exciting", "continue"
            ],
            "Finance": [
                # Finance-Alpaca specific terms
                "financial", "investment", "money", "profit", "market", "economic",
                "stock", "business", "trading", "portfolio", "risk", "return",
                "banking", "credit", "loan", "asset", "liability", "revenue", "cost"
            ]
        }

    def calculate_domain_confidence(self, text):
        """Calculate confidence scores for each domain based on updated keywords"""
        text_lower = text.lower()
        confidence_scores = {}

        for domain, keywords in self.domain_keywords.items():
            # Count keyword matches
            matches = sum(1 for keyword in keywords if keyword in text_lower)
            # Normalize by number of keywords and add bonus for multiple matches
            base_score = matches / len(keywords)
            bonus = min(matches * 0.1, 0.3)  # Bonus for multiple keyword matches
            confidence_scores[domain] = base_score + bonus

        return confidence_scores

    def route_to_best_adapter(self, prompt):
        """Route to best adapter with improved confidence calculation"""
        confidence_scores = self.calculate_domain_confidence(prompt)

        # Find best matching domain
        best_domain = max(confidence_scores.keys(), key=lambda k: confidence_scores[k])
        best_score = confidence_scores[best_domain]

        print(f"🎯 Routing Analysis for: '{prompt}'")
        for domain, score in confidence_scores.items():
            print(f"   {domain}: {score:.3f} confidence")


        # Lower threshold for better routing sensitivity
        if best_score > 0.05:  # Lower threshold for better sensitivity
            print(f"✅ Routing to {best_domain} adapter (confidence: {best_score:.3f})")
            return best_domain, best_score
        else:
            print("⚖️ Very low confidence - using Science adapter as default")
            return "Science", best_score

    def generate_smart_response(self, prompt, max_tokens=200):
        """Generate response using smart routing"""
        best_adapter, confidence = self.route_to_best_adapter(prompt)

        try:
            response = self.composer.test_single_adapter(best_adapter, prompt, max_tokens)
            return response, best_adapter, confidence
        except Exception as e:
            print(f"❌ Error with {best_adapter} adapter: {e}")
            # Fallback to Science adapter
            response = self.composer.test_single_adapter("Science", prompt, max_tokens)
            return response, "Science", confidence

In [ ]:
smart_router = SmartAdapterRouter(composer)

In [45]:
# Updated test prompts that match your dataset formats
smart_test_prompts = [
    # Science prompts (SciQ style)
    "Scientific Question: What is the process by which plants convert sunlight into energy?",
    "What happens when you mix an acid and a base in chemistry?",
    "Scientific Question: How do vaccines help prevent diseases?",

    # Creative prompts (TinyStories style)
    "Creative Prompt: Continue this story: Once upon a time, there was a little girl who found a magical key.",
    "Creative Prompt: Write a story about a friendly dragon who lived in a big castle.",
    "Creative Prompt: Tell me a story about two friends who went on an adventure.",

    # Finance prompts (Finance-Alpaca style)
    "Financial Query: What should I consider before investing in the stock market?",
    "Financial Query: How can I create a budget for my monthly expenses?",
    "Financial Query: What are the risks and benefits of cryptocurrency investment?",

    # Mixed/complex prompts
    "How might scientific research funding impact financial markets?",
    "Write a creative story about a scientist who discovers a way to predict stock prices."
]

print("🤖 Testing Smart Adapter Routing with Dataset-Specific Prompts")
print("=" * 70)

routing_results = []

for i, prompt in enumerate(smart_test_prompts, 1):
    print(f"\n🧪 Test {i}:")
    try:
        response, selected_adapter, confidence = smart_router.generate_smart_response(prompt)

        routing_results.append({
            'prompt': prompt,
            'selected_adapter': selected_adapter,
            'confidence': confidence,
            'response_length': len(response),
            'success': True
        })

        print(f"Selected: {selected_adapter} (confidence: {confidence:.3f})")
        print(f"Response: {response}")

    except Exception as e:
        print(f"❌ Error processing prompt: {e}")
        routing_results.append({
            'prompt': prompt,
            'selected_adapter': 'Error',
            'confidence': 0.0,
            'response_length': 0,
            'success': False
        })

    print("-" * 50)

successful_tests = [r for r in routing_results if r['success']]
print(f"\n📊 Routing Success Rate: {len(successful_tests)}/{len(routing_results)} ({len(successful_tests)/len(routing_results)*100:.1f}%)")

if successful_tests:
    avg_confidence = sum(r['confidence'] for r in successful_tests) / len(successful_tests)
    print(f"📈 Average Confidence: {avg_confidence:.3f}")


    adapter_counts = {}
    for result in successful_tests:
        adapter = result['selected_adapter']
        adapter_counts[adapter] = adapter_counts.get(adapter, 0) + 1

    print(f"🎯 Adapter Usage Distribution:")
    for adapter, count in adapter_counts.items():
        percentage = (count / len(successful_tests)) * 100
        print(f"   {adapter}: {count} times ({percentage:.1f}%)")

🤖 Testing Smart Adapter Routing with Dataset-Specific Prompts

🧪 Test 1:
🎯 Routing Analysis for: 'Scientific Question: What is the process by which ...'
   Science: 0.12 confidence
   Creative: 0.00 confidence
   Finance: 0.00 confidence
✅ Routing to Science adapter (confidence: 0.12)
Selected: Science (confidence: 0.125)
Response: Scientific Hypothesis: If plants receive an adequate amount of sunlight, then they will be able to convert that sunlight into energy.

Scientific Experiment: Create two identical plants in separate pots. Place one plant in a location with plenty of sunlight, and the other in a location with very little sunlight. Water both plants equally and observe their growth over a period of several weeks.

Scientific Conclusion: The plant that received more sunlight grew taller and had more leaves than the plant that received less sunlight. This supports our scientific hypothesis that plants can convert sunlight into energy.
---------------------------------------------